<a href="https://colab.research.google.com/github/sclatch1/recommenderSystems/blob/main/dataexploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
#!unzip -q cleaned_datasets_students.zip

^C


In [ ]:
#!pip install ydata-profiling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.7/398.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.7/679.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade recpack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 26.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4

In [ ]:
# importing libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.sparse import csr_matrix
import scipy as sp

from recpack.matrix import InteractionMatrix
from recpack.scenarios import WeakGeneralization
from recpack.util import get_top_K_ranks
from recpack.preprocessing.filters import MinItemsPerUser, MinUsersPerItem


from src.metrics import calculate_ndcg, calculate_calibrated_recall



In [108]:
# utility function to convert sparse matrix to dataframe
def matrix2df(X) -> pd.DataFrame:
    coo = sp.sparse.coo_array(X)
    return pd.DataFrame({
        "user_id": coo.row,
        "item_id": coo.col,
        "value": coo.data
    })

# helper function to convert a score matrix into a dataframe of recommendations
def scores2recommendations(
    scores: sp.sparse.csr_matrix, 
    X_test_in: sp.sparse.csr_matrix, 
    recommendation_count: int,
    prevent_history_recos = True
) -> pd.DataFrame:
    # ensure you don't recommend fold-in items
    if prevent_history_recos:
        scores[(X_test_in > 0)] = 0
    # rank items
    ranks = get_top_K_ranks(scores, recommendation_count)
    # convert to a dataframe
    df_recos = matrix2df(ranks).rename(columns={"value": "rank"}).sort_values(["user_id", "rank"])
    return df_recos

In [56]:
ROOT_DIR = Path.cwd()

data_dir_path = ROOT_DIR / "data"

/home/david/Desktop/recommenderSystems/data


## Datastructure
    - type of data: catergorical, numerical, text, dates
    - remove duplicates and irrelevan features
    - datatypes align with the content ?

In [57]:
# load seperatly -> very heavy
item_reviews = pd.read_csv(f"{data_dir_path}/item_reviews.csv")

In [58]:
games = pd.read_csv(f"{data_dir_path}/games.csv")
user_reviews = pd.read_csv(f"{data_dir_path}/user_reviews.csv")
train_interaction = pd.read_csv(f"{data_dir_path}/train_interactions.csv")
extended_games = pd.read_csv(f"{data_dir_path}/extended_games.csv")
test_interaction = pd.read_csv(f"{data_dir_path}/test_interactions_in.csv")


In [59]:
# Looking for relevant columns

print("games columns")
print(games.columns)
print("user_reviews columns")
print(user_reviews.columns)
print("train interactions columns")
print(train_interaction.columns)
print("extended games columns")
print(extended_games.columns)

games columns
Index(['item_id', 'item_name', 'publisher', 'genres', 'url', 'tags',
       'sentiment', 'metascore', 'specs', 'price', 'release_date'],
      dtype='object')
user_reviews columns
Index(['user_id', 'item_id', 'funny', 'posted', 'last_edited', 'helpful',
       'recommend', 'review'],
      dtype='object')
train interactions columns
Index(['user_id', 'item_id', 'item_name', 'playtime'], dtype='object')
extended games columns
Index(['item_id', 'item_name', 'release_date', 'required_age', 'price',
       'dlc_count', 'detailed_description', 'about_the_game',
       'short_description', 'reviews',
       ...
       'tags_Shop Keeper', 'tags_Coding', 'tags_Football (Soccer)',
       'tags_Hobby Sim', 'tags_Tile-Matching', 'tags_Mahjong', 'tags_Birds',
       'tags_Football (American)', 'tags_Fox', 'tags_Extraction Shooter'],
      dtype='object', length=490)


We want to evaluate ItemKNN, which is based on item-user interaction. So we will not use the following datasets: games, extended games.

In [60]:
# remove the duplicates record
train_interaction.drop_duplicates(inplace=True)

In [61]:
from ydata_profiling import ProfileReport
profile = ProfileReport(train_interaction, title="Steam Dataset Report", explorative=True)
profile.to_file(f"{ROOT_DIR}/reports/train_interaction_report")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [62]:
user_reviews.drop_duplicates(inplace=True)
user_reviews.head()

,user_id,item_id,funny,posted,last_edited,helpful,recommend,review
0,0,140,NaN,"Posted November 5, 2011.",NaN,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,711,NaN,"Posted July 15, 2011.",NaN,No ratings yet,True,It's unique and worth a playthrough.
2,1,815,NaN,"Posted September 8, 2013.",NaN,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
3,1,1027,NaN,"Posted November 29, 2013.",NaN,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...
4,2,1224,NaN,Posted February 3.,NaN,No ratings yet,True,A suitably punishing roguelike platformer. Wi...


We can remove the following collumns review, helpful, funny and last edited

In [ ]:
user_reviews.columns

Index(['user_id', 'item_id', 'funny', 'posted', 'last_edited', 'helpful',
       'recommend', 'review'],
      dtype='object')

In [63]:
user_reviews.drop(columns=['posted', 'helpful', 'funny', 'review', 'last_edited'], inplace=True)

In [ ]:
profile = ProfileReport(user_reviews, title="Steam Dataset Report", explorative=True)
profile.to_file(f"{ROOT_DIR}/reports/user_reviews_report")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 101.69it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/home/david/Desktop/recommenderSystems/.venv/lib/python3.11/site-packages/ydata_profiling/profile_report.py:386: UserWarning: Extension  not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Let's first build an interactionMatrix with train_interaction dataset

In [188]:

unique_users_before = train_interaction['user_id'].nunique()
unique_items_before = train_interaction['item_id'].nunique()

print(f"Unique users before filtering: {unique_users_before}")
print(f"Unique items before filtering: {unique_items_before}")

print("Sample user IDs before filtering:", train_interaction['user_id'].unique()[:10])
print("Sample item IDs before filtering:", train_interaction['item_id'].unique()[:10])

f1 = MinUsersPerItem(
    min_users_per_item=5,
    user_ix="user_id",
    item_ix="item_id"
)

train_interaction = f1.apply(train_interaction)

f2 = MinItemsPerUser(
    min_items_per_user=10,
    user_ix="user_id",
    item_ix="item_id"
)

train_interaction = f2.apply(train_interaction)


unique_users_after = train_interaction['user_id'].nunique()
unique_items_after = train_interaction['item_id'].nunique()

print(f"Unique users after filtering: {unique_users_after}")
print(f"Unique items after filtering: {unique_items_after}")

print("Sample user IDs after filtering:", train_interaction['user_id'].unique()[:10])
print("Sample item IDs after filtering:", train_interaction['item_id'].unique()[:10])

train_interaction['user_idx'] = pd.factorize(train_interaction['user_id'])[0]
train_interaction['item_idx'] = pd.factorize(train_interaction['item_id'])[0]

X = InteractionMatrix(train_interaction, item_ix="item_id", user_ix="user_id")

# Count unique items per user
user_interaction_counts = train_interaction.groupby('user_id')['item_id'].nunique()
print("Min interactions per user:", user_interaction_counts.min())
print("Median interactions per user:", user_interaction_counts.median())
print("Max interactions per user:", user_interaction_counts.max())

# Optional: check number of "cold users"
cold_users_5 = (user_interaction_counts < 5).sum()
cold_users_10 = (user_interaction_counts < 10).sum()
print("Users with <5 interactions:", cold_users_5)
print("Users with <10 interactions:", cold_users_10)



Unique users before filtering: 40794
Unique items before filtering: 6274
Sample user IDs before filtering: [ 0  1  2  3  5  8  9 10 11 12]
Sample item IDs before filtering: [   0 2555 2556 1043 5335 1592 5584 5653 8285 8324]
Unique users after filtering: 40794
Unique items after filtering: 6274
Sample user IDs after filtering: [ 0  1  2  3  5  8  9 10 11 12]
Sample item IDs after filtering: [   0 2555 2556 1043 5335 1592 5584 5653 8285 8324]
Min interactions per user: 10
Median interactions per user: 37.0
Max interactions per user: 2721
Users with <5 interactions: 0
Users with <10 interactions: 0


Weak generalization is more practical. For some algorithms (e.g. matrix
factorization), a strong generalization data split is infeasible as the
algorithm relies on learned user representations.

In [215]:
# fix seed for reproducibility
scenario = WeakGeneralization(frac_data_in=0.8, validation=True, seed=42)
scenario.split(X)



X_test_in = scenario.test_data_in.values
X_test_out = scenario.test_data_out.values

X_val_train, X_val_target = scenario.validation_data
val_train = X_val_train.values
val_target = X_val_target.values

df_test_out = matrix2df(X_test_out)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [216]:
# -----------------------------
# 1️⃣ Filter original DataFrame
# -----------------------------
train_interaction = MinUsersPerItem(
    min_users_per_item=5, user_ix="user_id", item_ix="item_id"
).apply(train_interaction)

train_interaction = MinItemsPerUser(
    min_items_per_user=10, user_ix="user_id", item_ix="item_id"
).apply(train_interaction)

print("Unique users after filtering:", train_interaction['user_id'].nunique())
print("Unique items after filtering:", train_interaction['item_id'].nunique())

# Factorize IDs to remove gaps (important for InteractionMatrix)
train_interaction['user_idx'] = pd.factorize(train_interaction['user_id'])[0]
train_interaction['item_idx'] = pd.factorize(train_interaction['item_id'])[0]

# -----------------------------
# 2️⃣ Build InteractionMatrix
# -----------------------------
X = InteractionMatrix(train_interaction, user_ix='user_idx', item_ix='item_idx')

# -----------------------------
# 3️⃣ Split with WeakGeneralization
# -----------------------------
scenario = WeakGeneralization(frac_data_in=0.8, validation=True, seed=42)
scenario.split(X)

# Get matrices
X_train = scenario.full_training_data.values
X_test_in = scenario.test_data_in.values
X_test_out = scenario.test_data_out.values
X_val_train, X_val_target = scenario.validation_data
val_train = X_val_train.values
val_target = X_val_target.values

# -----------------------------
# 5️⃣ Sanity check
# -----------------------------
print("Train shape:", X_train.shape)
print("Validation shape:", val_train_clean.shape)
print("Test shape:", X_test_out_clean.shape)
print("Train nnz:", X_train.nnz)
print("Validation nnz:", val_target_clean.nnz)
print("Test nnz:", X_test_out_clean.nnz)

Unique users after filtering: 40794
Unique items after filtering: 6274


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Train shape: (40794, 6274)
Validation shape: (40794, 6274)
Test shape: (40794, 6274)
Train nnz: 1779966
Validation nnz: 339617
Test nnz: 425143


In [217]:
def inspect_dataset(X_train, X_val_train, X_val_target, X_test_in=None):
    import numpy as np
    
    # --- Safety: unpack if given a tuple ---
    if isinstance(X_val_train, tuple):
        print("Unpacking X_val_train tuple...")
        X_val_train, X_val_target = X_val_train

    # --- Safety: check sparse matrix attributes ---
    def nnz_safe(mat):
        return mat.nnz if hasattr(mat, "nnz") else None

    print("===== Dataset Shapes =====")
    print(f"Train interaction matrix:        {X_train.shape}")
    if X_test_in is not None:
        print(f"Test (in) interaction matrix:    {X_test_in.shape}")
    print(f"Validation train interactions:   nnz = {nnz_safe(X_val_train)}")
    print(f"Validation target interactions:  nnz = {nnz_safe(X_val_target)}")
    print()

    train_counts = np.diff(X_train.indptr)

    print("===== Training Interaction Statistics =====")
    print(f"Mean interactions per user:      {train_counts.mean():.3f}")
    print(f"Median interactions per user:    {np.median(train_counts):.3f}")
    print(f"Min interactions per user:       {train_counts.min()}")
    print(f"Max interactions per user:       {train_counts.max()}")
    print()

    cold_1 = np.sum(train_counts < 1)
    cold_5 = np.sum(train_counts < 5)
    cold_10 = np.sum(train_counts < 10)

    print("===== Cold-User Counts =====")
    print(f"Users with < 1 interaction:      {cold_1}")
    print(f"Users with < 5 interactions:     {cold_5}")
    print(f"Users with < 10 interactions:    {cold_10}")
    print()

    return {
        "mean": train_counts.mean(),
        "median": np.median(train_counts),
        "cold_under_5": cold_5,
        "cold_under_10": cold_10,
    }


stats = inspect_dataset(X_train, val_train, val_target, X_test_in)


===== Dataset Shapes =====
Train interaction matrix:        (40794, 6274)
Test (in) interaction matrix:    (40794, 6274)
Validation train interactions:   nnz = 1440350
Validation target interactions:  nnz = 339617

===== Training Interaction Statistics =====
Mean interactions per user:      43.633
Median interactions per user:    30.000
Min interactions per user:       8
Max interactions per user:       2177

===== Cold-User Counts =====
Users with < 1 interaction:      0
Users with < 5 interactions:     0
Users with < 10 interactions:    2048



In [218]:
# Interactions per user in test set
test_counts = X_test_out_clean.sum(axis=1)
print("Median test interactions per user:", np.median(test_counts))
print("Min test interactions per user:", test_counts.min())
print("Max test interactions per user:", test_counts.max())


Median test interactions per user: [[ 4.  2.  2. ... 24. 24. 34.]]
Min test interactions per user: 2
Max test interactions per user: 544


In [219]:
import os
os.chdir("/home/david/Desktop/recommenderSystems")
import importlib
from src import mfbpr

importlib.reload(mfbpr)

from src.mfbpr import PPAC_BPRMF

model = PPAC_BPRMF(
    num_components=100,
    learning_rate=0.01,
    lambda_w=0.001,
    lambda_h=0.001,
    validation_sample_size=100,
    predict_topK=100,
    save_best_to_file=f"{ROOT_DIR}/models/best_model.pkl"
)

model._init_model(X_train)



Initializing PPAC_BPRMF
1000


In [220]:
model.fit(X_train, validation_data=(val_train_clean, val_target_clean))


1000


/home/david/Desktop/recommenderSystems/src/mfbpr.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/david/Desktop/recommenderSystems/src/mfbpr.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._ppac_weights[pos_items.cpu()], device=self.device


2025-12-02 21:37:06,508 - base - recpack - INFO - Processed epoch 0 in 7.04 s.Batch Training Loss = 0.1117


/home/david/Desktop/recommenderSystems/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:201: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray_sparse(i, j, x)


2025-12-02 21:37:07,545 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.6931471831859948, which is better than previous iterations.
2025-12-02 21:37:07,546 - base - recpack - INFO - Model improved. Storing better model.
2025-12-02 21:37:07,568 - base - recpack - INFO - Evaluation at end of 0 took 1.06 s.
2025-12-02 21:37:13,197 - base - recpack - INFO - Processed epoch 1 in 5.63 s.Batch Training Loss = 0.1103
2025-12-02 21:37:13,880 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.6931471825447452, which is worse than previous iterations.
2025-12-02 21:37:13,881 - base - recpack - INFO - Evaluation at end of 1 took 0.68 s.
2025-12-02 21:37:19,893 - base - recpack - INFO - Processed epoch 2 in 6.01 s.Batch Training Loss = 0.1103
2025-12-02 21:37:20,631 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.6931471822863278, which is worse than previous iterations.
2025-12-02 21:37:20,632 - base - recpack - INFO - Evaluation at en

,num_components,100
,lambda_h,0.001
,lambda_w,0.001
,batch_size,1000
,max_epochs,20
,learning_rate,0.01
,stopping_criterion,<recpack.algo...x74e23e6b65d0>
,stop_early,False
,max_iter_no_change,5
,min_improvement,0.01
,seed,1072136351


In [221]:
scores = model.predict(X_test_in_clean)


In [222]:
df_recos = scores2recommendations(scores, X_test_in, 10)

df_recos.drop(columns=['rank'], inplace=True)
df_recos.to_csv(f"{ROOT_DIR}/reports/df_recos.csv", index=False)


/home/david/Desktop/recommenderSystems/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:210: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)


In [223]:
ndcg = calculate_ndcg(df_recos, 10, df_test_out)
recall = calculate_calibrated_recall(df_recos, 10, df_test_out)

print(f"  NDCG@10: {ndcg:.5f}")
print(f"Recall@10: {recall:.5f}")

  NDCG@10: 0.00053
Recall@10: 0.00059


In [156]:
import numpy as np
user_interactions = np.diff(X_train.tocsr().indptr)
print("Mean interactions per user:", user_interactions.mean())
print("Median:", np.median(user_interactions))
print("Users with < 5 interactions:", (user_interactions < 5).sum())


Mean interactions per user: 26.535655276279044
Median: 13.0
Users with < 5 interactions: 23022


In [ ]:

# ---------------------------
# 1. Compute "item length" baseline (median playtime per item)
# ---------------------------
item_median = train_interaction.groupby("item_id")["playtime"].median()
df = train_interaction.merge(item_median.rename("item_median"), on="item_id", how="left")

# Avoid divide-by-zero
df["item_median"].replace(0, np.nan, inplace=True)
df["item_median"].fillna(df["item_median"].median(), inplace=True)

# ---------------------------
# 2. Normalize playtime by typical item duration
# ---------------------------
df["playtime_norm"] = df["playtime"] / df["item_median"]

# ---------------------------
# 3. Apply log transform
# ---------------------------
df["weight"] = np.log1p(df["playtime_norm"])

/tmp/ipython-input-4134867206.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["item_median"].replace(0, np.nan, inplace=True)
/tmp/ipython-input-4134867206.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

In [ ]:
# Keep category dtypes
user_cat = df["user_id"].astype("category")
item_cat = df["item_id"].astype("category")

df["user_idx"] = user_cat.cat.codes
df["item_idx"] = item_cat.cat.codes

num_users = len(user_cat.cat.categories)
num_items = len(item_cat.cat.categories)

# Build TRAIN matrix
matrix = csr_matrix(
    (df["weight"], (df["user_idx"], df["item_idx"])),
    shape=(num_users, num_items)
)

In [ ]:
from src.mfbpr import PPAC_BPRMF

model = PPAC_BPRMF(
    num_components=100,
    learning_rate=0.01,
    lambda_w=0.001,
    lambda_h=0.001
)

model.fit(matrix)

2025-11-25 14:28:21,993 - base - recpack - INFO - Fitting ItemKNN complete - Took 6.84s


INFO:recpack:Fitting ItemKNN complete - Took 6.84s


ItemKNN()

In [ ]:
# ---------------------------
# TEST DATA
# ---------------------------
test = test_interaction.copy()

# Merge training medians into test
test = test.merge(item_median.rename("item_median"), on="item_id", how="left")

# For unseen items in test: use global median (keeps transformation stable)
fallback = item_median.median()
test["item_median"] = test["item_median"].fillna(fallback)

# Normalize playtime
test["playtime_norm"] = test["playtime"] / test["item_median"]

# Log transform
test["weight"] = np.log1p(test["playtime_norm"])


In [ ]:
test_to_predict = test_interaction[["user_id", "item_id"]].copy()
test_to_predict.to_csv("test_to_predict.csv", index=False)




In [ ]:
# Encode users and items using training categories
test_to_predict["user_idx"] = (
    test_to_predict["user_id"]
    .astype("category")
    .cat.set_categories(user_cat.cat.categories)
    .cat.codes
)

test_to_predict["item_idx"] = (
    test_to_predict["item_id"]
    .astype("category")
    .cat.set_categories(item_cat.cat.categories)
    .cat.codes
)

# Remove completely unseen users/items
test_to_predict = test_to_predict[
    (test_to_predict["user_idx"] != -1) &
    (test_to_predict["item_idx"] != -1)
]

# Build prediction mask (values = 1)
prediction_matrix = csr_matrix(
    (np.ones(len(test_to_predict)),
     (test_to_predict["user_idx"], test_to_predict["item_idx"])),
    shape=(num_users, num_items)
)



In [ ]:
pred = itemknn.predict(prediction_matrix)
print(pred)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 0 stored elements and shape (54315, 8368)>


In [ ]:
# 1️⃣ Convert CSR to COO (easy for iteration)
coo = pred.tocoo()

# 2️⃣ Map indices back to original IDs
user_ids = [user_mapping[i] for i in coo.row]
item_ids = [item_mapping[i] for i in coo.col]
values = coo.data

# 3️⃣ Build a DataFrame
df_pred = pd.DataFrame({
    "user_id": user_ids,
    "item_id": item_ids,
    "score": values
})

df_pred.head()


,user_id,item_id,score
